In [ ]:
import os
import requests
import pandas as pd
from tqdm.auto import tqdm

from db import db

# Выгрузка и сохранение категорий

In [ ]:
API_KEY = ''
BASE_URL = 'https://mpstats.io/api/'

HEADERS = {
    'X-Mpstats-TOKEN': API_KEY,
    'Content-Type': 'application/json'
    }
url_param = 'user/report_api_limit'
response = requests.get(
    BASE_URL+url_param,
    headers=HEADERS
)
if response.status_code == 200:
    check_limit = response.json()
    print(check_limit)
else: 
    print(response.status_code)
    print(response.text)

{'available': 4000, 'use': 3227}


In [3]:
def load_trend(category,d1, d2):
    url_param = f'wb/get/category/by_date'

    api_params = {
        'd2': d2,
        'd1': d1,
        'path': category,
        'groupBy': 'week'
    }
    response = requests.get(
        url=BASE_URL+url_param,
        params=api_params,
        headers=HEADERS,
    )
    data = pd.DataFrame(response.json())

    return data

In [27]:
categories = db.get_table_by_request(
    """SELECT "Артикул", "Категория_wb", "Вариации", status, Группа
    FROM ir_db.mp_stats_categories AS cat
    JOIN ir_db.Реестр AS rst
    ON cat.Артикул = rst.art
    WHERE rst.status != 'архив'""")
categories.columns = ["art", "category", "variation", "status", "Группа"]

In [28]:
# тут компануем финальную категорию, в соответствии с группой
categories['Группа'] = categories['Группа'].apply(lambda x: x if x!='' else None)
categories['category_final'] = categories['Группа']
categories.loc[categories['category_final'].isna(), 'category_final'] = categories.loc[categories['category_final'].isna(),'category']

In [30]:
categories['category_final'].unique()

array(['Аксессуары/Чехлы для планшетов',
       'Канцтовары/Офисные принадлежности/Нож канцелярский',
       'Дом/Хранение вещей/Органайзеры/Разделитель для ящика',
       'Раздвижная полка',
       'Дом/Предметы интерьера/Свечи и подсвечники/Подсвечник',
       'Дом/Кухня/Столовые приборы', 'Дом/Кухня/Кружки/Керамические',
       'Канцтовары/Бумажная продукция/Ежедневник',
       'Красота/Аксессуары/Расчески и бигуди/Бигуди',
       'Канцтовары/Офисные принадлежности/Органайзер для сменных блоков',
       'Электроника/Ноутбуки и компьютеры/Внешние накопители информации/Flash-накопитель',
       'Аксессуары/Сумки и рюкзаки/Папки-сумки',
       'Канцтовары/Письменные принадлежности/Маркер',
       'Канцтовары/Офисные принадлежности/Подставка канцелярская',
       'Канцтовары/Письменные принадлежности/Пенал', None,
       'Электроника/Гарнитуры и наушники/Чехол для наушников',
       'Беспроводные зарядки',
       'Электроника/Ноутбуки и компьютеры/Периферийные аксессуары/Микрофон',
    

In [32]:
d1 = '2020-06-01'
d2 = '2024-04-01'
for category in tqdm(categories['category_final'].unique()):
    if category is None:
        continue
    name = category.replace('/','-')
    if os.path.lexists(f'data/categories/{name}.csv'):
        continue
    data = load_trend(category, d1, d2)
    if data.empty:
        print(category)
        print('Нет данных! Проверьте корректность названия категории')
        continue
        
    data.to_csv(f'data/categories/{name}.csv')
    

  0%|          | 0/1 [00:00<?, ?it/s]

Аксессуары/Чехлы для планшетов
Нет данных! Проверьте корректность названия категории
